In [1]:
from tensorflow.keras.layers import MultiHeadAttention
import io
import re
import string
import tqdm

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

In [3]:
from tensorflow.keras.datasets import imdb
import numpy as np

In [2]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

In [4]:
sentence = "The wide road shimmered in the hot sun"
tokens = list(sentence.lower().split())
print(len(tokens))

8


In [5]:
vocab, index = {}, 1  # start indexing from 1
vocab['<pad>'] = 0  # add a padding token
for token in tokens:
  if token not in vocab:
    vocab[token] = index
    index += 1
vocab_size = len(vocab)
print(vocab)

{'<pad>': 0, 'the': 1, 'wide': 2, 'road': 3, 'shimmered': 4, 'in': 5, 'hot': 6, 'sun': 7}


In [6]:
inverse_vocab = {index: token for token, index in vocab.items()}
print(inverse_vocab)

{0: '<pad>', 1: 'the', 2: 'wide', 3: 'road', 4: 'shimmered', 5: 'in', 6: 'hot', 7: 'sun'}


In [7]:
example_sequence = [vocab[word] for word in tokens]
print(example_sequence)

[1, 2, 3, 4, 5, 1, 6, 7]


In [8]:
window_size = 2
positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
      example_sequence,
      vocabulary_size=vocab_size,
      window_size=window_size,
      negative_samples=0)
print(len(positive_skip_grams))

26


In [10]:
for target, context in positive_skip_grams[:5]:
  print(f"({target}, {context}): ({inverse_vocab[target]}, {inverse_vocab[context]})")

(6, 1): (hot, the)
(4, 3): (shimmered, road)
(3, 4): (road, shimmered)
(1, 2): (the, wide)
(1, 6): (the, hot)


In [18]:
target_word, context_word = positive_skip_grams[0]
num_ns = 4
context_class = tf.reshape(tf.constant(context_word, dtype="int64"), (1, 1))
negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes=context_class,  # class that should be sampled as 'positive'
    num_true=1,  # each positive skip-gram has 1 positive context class
    num_sampled=num_ns,  # number of negative context words to sample
    unique=True,  # all the negative samples should be unique
    range_max=vocab_size,  # pick index of the samples from [0, vocab_size]
    seed=SEED,  # seed for reproducibility
    name="negative_sampling"  # name of this operation
)
print(negative_sampling_candidates)
print([inverse_vocab[index.numpy()] for index in negative_sampling_candidates])

tf.Tensor([2 1 4 3], shape=(4,), dtype=int64)
['wide', 'the', 'shimmered', 'road']


In [19]:
# Reduce a dimension so you can use concatenation (in the next step).
squeezed_context_class = tf.squeeze(context_class, 1)

# Concatenate a positive context word with negative sampled words.
context = tf.concat([squeezed_context_class, negative_sampling_candidates], 0)

# Label the first context word as `1` (positive) followed by `num_ns` `0`s (negative).
label = tf.constant([1] + [0]*num_ns, dtype="int64")
target = target_word

In [20]:
print(f"target_index    : {target}")
print(f"target_word     : {inverse_vocab[target_word]}")
print(f"context_indices : {context}")
print(f"context_words   : {[inverse_vocab[c.numpy()] for c in context]}")
print(f"label           : {label}")

target_index    : 6
target_word     : hot
context_indices : [1 2 1 4 3]
context_words   : ['the', 'wide', 'the', 'shimmered', 'road']
label           : [1 0 0 0 0]


In [21]:
print("target  :", target)
print("context :", context)
print("label   :", label)

target  : 6
context : tf.Tensor([1 2 1 4 3], shape=(5,), dtype=int64)
label   : tf.Tensor([1 0 0 0 0], shape=(5,), dtype=int64)


In [15]:
target_word,context_word

(6, 1)

In [22]:
sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(size=10)
print(sampling_table)

[0.00315225 0.00315225 0.00547597 0.00741556 0.00912817 0.01068435
 0.01212381 0.01347162 0.01474487 0.0159558 ]


In [23]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for `vocab_size` tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in the dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with a positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=seed,
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      context = tf.concat([tf.squeeze(context_class,1), negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

In [24]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


In [25]:
with open(path_to_file) as f:
  lines = f.read().splitlines()
for line in lines[:20]:
  print(line)

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.


In [26]:
text_ds = tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x), bool))

In [28]:
# Now, create a custom standardization function to lowercase the text and
# remove punctuation.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')


# Define the vocabulary size and the number of words in a sequence.
vocab_size = 4096
sequence_length = 10

# Use the `TextVectorization` layer to normalize, split, and map strings to
# integers. Set the `output_sequence_length` length to pad all samples to the
# same length.
vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

In [29]:
vectorize_layer.adapt(text_ds.batch(1024))

In [30]:
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

['', '[UNK]', 'the', 'and', 'to', 'i', 'of', 'you', 'my', 'a', 'that', 'in', 'is', 'not', 'for', 'with', 'me', 'it', 'be', 'your']


In [31]:
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

In [32]:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))

32777


In [33]:
for seq in sequences[:5]:
  print(f"{seq} => {[inverse_vocab[i] for i in seq]}")

[ 89 270   0   0   0   0   0   0   0   0] => ['first', 'citizen', '', '', '', '', '', '', '', '']
[138  36 982 144 673 125  16 106   0   0] => ['before', 'we', 'proceed', 'any', 'further', 'hear', 'me', 'speak', '', '']
[34  0  0  0  0  0  0  0  0  0] => ['all', '', '', '', '', '', '', '', '', '']
[106 106   0   0   0   0   0   0   0   0] => ['speak', 'speak', '', '', '', '', '', '', '', '']
[ 89 270   0   0   0   0   0   0   0   0] => ['first', 'citizen', '', '', '', '', '', '', '', '']


In [34]:
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=2,
    num_ns=4,
    vocab_size=vocab_size,
    seed=SEED)

targets = np.array(targets)
contexts = np.array(contexts)
labels = np.array(labels)

print('\n')
print(f"targets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")

100%|██████████| 32777/32777 [00:15<00:00, 2111.71it/s]




targets.shape: (65204,)
contexts.shape: (65204, 5)
labels.shape: (65204, 5)


In [41]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

<_BatchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>


In [42]:
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

<_PrefetchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>


In [43]:
class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(vocab_size,
                                      embedding_dim,
                                      input_length=1,
                                      name="w2v_embedding")
    self.context_embedding = layers.Embedding(vocab_size,
                                       embedding_dim,
                                       input_length=num_ns+1)

  def call(self, pair):
    target, context = pair
    # target: (batch, dummy?)  # The dummy axis doesn't exist in TF2.7+
    # context: (batch, context)
    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    # target: (batch,)
    word_emb = self.target_embedding(target)
    # word_emb: (batch, embed)
    context_emb = self.context_embedding(context)
    # context_emb: (batch, context, embed)
    dots = tf.einsum('be,bce->bc', word_emb, context_emb)
    # dots: (batch, context)
    return dots

In [74]:
num_ns


4

In [55]:
target,context = dataset

ValueError: too many values to unpack (expected 2)

In [44]:
def custom_loss(x_logit, y_true):
      return tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=y_true)

In [71]:
embedding_dim

128

In [45]:
embedding_dim = 128
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

In [46]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [56]:
hsitory = word2vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])

Epoch 1/20
 1/63 [..............................] - ETA: 1s - loss: 0.4416 - accuracy: 0.9043

63/63 [==============================] - 1s 8ms/step - loss: 0.4412 - accuracy: 0.9037
Epoch 2/20
63/63 [==============================] - 0s 8ms/step - loss: 0.4160 - accuracy: 0.9106
Epoch 3/20
63/63 [==============================] - 0s 8ms/step - loss: 0.3930 - accuracy: 0.9165
Epoch 4/20
63/63 [==============================] - 0s 8ms/step - loss: 0.3720 - accuracy: 0.9217
Epoch 5/20
63/63 [==============================] - 1s 8ms/step - loss: 0.3527 - accuracy: 0.9267
Epoch 6/20
63/63 [==============================] - 0s 8ms/step - loss: 0.3351 - accuracy: 0.9311
Epoch 7/20
63/63 [==============================] - 0s 8ms/step - loss: 0.3189 - accuracy: 0.9351
Epoch 8/20
63/63 [==============================] - 0s 8ms/step - loss: 0.3041 - accuracy: 0.9386
Epoch 9/20
63/63 [==============================] - 1s 8ms/step - loss: 0.2904 - accuracy: 0.9418
Epoch 10/20
63/63 [==============================] - 1s 8ms/step - loss: 0.2779 - accuracy: 0.9446
Epoch 11/20
63/63 [===========

In [48]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [50]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [78]:
import gensim

In [ ]:
f = open('vectors.txt' ,'w')
f.write('{} {}\n'.format(vocab_size-1, embed_size))
vectors = model.get_weights()[0]
for word, i in tokenizer.word_index.items():
    f.write('{} {}\n'.format(word, ' '.join(map(str, list(vectors[i, :])))))
f.close()

# 모델 로드
w2v = gensim.models.KeyedVectors

In [77]:
word2vec.most_similar(positive = ['the'])

AttributeError: 'Word2Vec' object has no attribute 'most_similar'

In [79]:
model_fname = 'D:/Data/embedding/data/word-embeddings/word2vec/word2vec'

In [80]:
model_fname

'D:/Data/embedding/data/word-embeddings/word2vec/word2vec'

In [97]:
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
from tqdm import tqdm

corpus_fname = path_to_file


class callback(CallbackAny2Vec):
    """Callback to print loss after each epoch."""

    def __init__(self):
        self.epoch = 0
        self.loss_to_be_subed = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        loss_now = loss - self.loss_to_be_subed
        self.loss_to_be_subed = loss
        print('Loss after epoch {}: {}'.format(self.epoch, loss_now))
        self.epoch += 1


print('corpus 생성')
corpus = [sent.strip().split(" ") for sent in tqdm(open(corpus_fname, 'r', encoding='utf-8').readlines())]

print("학습 중")
model = Word2Vec(corpus, vector_size=100, workers=4, sg=1, compute_loss=True, epochs=50, callbacks=[callback()])
print('완료')

corpus 생성


100%|██████████| 40000/40000 [00:00<00:00, 203005.86it/s]


학습 중
Loss after epoch 0: 385946.53125
Loss after epoch 1: 305597.65625
Loss after epoch 2: 302774.9375
Loss after epoch 3: 295702.875
Loss after epoch 4: 283588.875
Loss after epoch 5: 289524.5
Loss after epoch 6: 285855.875
Loss after epoch 7: 280258.5
Loss after epoch 8: 266780.75
Loss after epoch 9: 264066.25
Loss after epoch 10: 265936.75
Loss after epoch 11: 264048.5
Loss after epoch 12: 254998.5
Loss after epoch 13: 249148.5
Loss after epoch 14: 248400.0
Loss after epoch 15: 221302.5
Loss after epoch 16: 223013.5
Loss after epoch 17: 217316.5
Loss after epoch 18: 220591.0
Loss after epoch 19: 182993.0
Loss after epoch 20: 217501.5
Loss after epoch 21: 216739.0
Loss after epoch 22: 214309.5
Loss after epoch 23: 213081.5
Loss after epoch 24: 212797.0
Loss after epoch 25: 181418.0
Loss after epoch 26: 212836.5
Loss after epoch 27: 213270.5
Loss after epoch 28: 205629.5
Loss after epoch 29: 208464.0
Loss after epoch 30: 176357.0
Loss after epoch 31: 208647.5
Loss after epoch 32: 1753

In [129]:
corpus

[['First', 'Citizen:'],
 ['Before', 'we', 'proceed', 'any', 'further,', 'hear', 'me', 'speak.'],
 [''],
 ['All:'],
 ['Speak,', 'speak.'],
 [''],
 ['First', 'Citizen:'],
 ['You',
  'are',
  'all',
  'resolved',
  'rather',
  'to',
  'die',
  'than',
  'to',
  'famish?'],
 [''],
 ['All:'],
 ['Resolved.', 'resolved.'],
 [''],
 ['First', 'Citizen:'],
 ['First,',
  'you',
  'know',
  'Caius',
  'Marcius',
  'is',
  'chief',
  'enemy',
  'to',
  'the',
  'people.'],
 [''],
 ['All:'],
 ['We', "know't,", 'we', "know't."],
 [''],
 ['First', 'Citizen:'],
 ['Let',
  'us',
  'kill',
  'him,',
  'and',
  "we'll",
  'have',
  'corn',
  'at',
  'our',
  'own',
  'price.'],
 ["Is't", 'a', 'verdict?'],
 [''],
 ['All:'],
 ['No',
  'more',
  'talking',
  "on't;",
  'let',
  'it',
  'be',
  'done:',
  'away,',
  'away!'],
 [''],
 ['Second', 'Citizen:'],
 ['One', 'word,', 'good', 'citizens.'],
 [''],
 ['First', 'Citizen:'],
 ['We', 'are', 'accounted', 'poor', 'citizens,', 'the', 'patricians', 'good.'],
 ['

In [114]:
open(corpus_fname, 'r', encoding='utf-8').readlines()

['First Citizen:\n',
 'Before we proceed any further, hear me speak.\n',
 '\n',
 'All:\n',
 'Speak, speak.\n',
 '\n',
 'First Citizen:\n',
 'You are all resolved rather to die than to famish?\n',
 '\n',
 'All:\n',
 'Resolved. resolved.\n',
 '\n',
 'First Citizen:\n',
 'First, you know Caius Marcius is chief enemy to the people.\n',
 '\n',
 'All:\n',
 "We know't, we know't.\n",
 '\n',
 'First Citizen:\n',
 "Let us kill him, and we'll have corn at our own price.\n",
 "Is't a verdict?\n",
 '\n',
 'All:\n',
 "No more talking on't; let it be done: away, away!\n",
 '\n',
 'Second Citizen:\n',
 'One word, good citizens.\n',
 '\n',
 'First Citizen:\n',
 'We are accounted poor citizens, the patricians good.\n',
 'What authority surfeits on would relieve us: if they\n',
 'would yield us but the superfluity, while it were\n',
 'wholesome, we might guess they relieved us humanely;\n',
 'but they think we are too dear: the leanness that\n',
 'afflicts us, the object of our misery, is as an\n',
 'in

In [105]:
model.raw_vocab

defaultdict(int, {})

In [108]:
word_vectors = model.wv

In [127]:
word_vectors.most_similar('Citizen:')

[('Servingman:', 0.9981366991996765),
 ('Murderer:', 0.9971839785575867),
 ('Gentleman:', 0.9959954023361206),
 ('Servant:', 0.9959403276443481),
 ('Conspirator:', 0.9944868087768555),
 ('Watchman:', 0.9939625859260559),
 ('Messenger:', 0.9933732748031616),
 ('Keeper:', 0.9931440949440002),
 ('Senator:', 0.9927027225494385),
 ('Officer:', 0.9924015402793884)]

In [84]:
 tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

'C:\\Users\\pop75\\.keras\\datasets\\shakespeare.txt'

In [135]:
from SmilesPE.pretokenizer import kmer_tokenizer
import sys
smi = 'CC[N+](C)(C)Cc1ccccc1Br'
toks = kmer_tokenizer(smi, ngram=4)
print(toks)

['CC[N+](', 'C[N+](C', '[N+](C)', '(C)(', 'C)(C', ')(C)', '(C)C', 'C)Cc', ')Cc1', 'Cc1c', 'c1cc', '1ccc', 'cccc', 'cccc', 'ccc1', 'cc1Br']


In [136]:
from TDC.tdc.single_pred import Tox

In [171]:
train, label = Tox(name='clintox').get_data(format='DeepPurpose')

Found local copy...
Loading...
Done!


In [173]:
train

array(['*C(=O)[C@H](CCCCNC(=O)OCCOC)NC(=O)OCCOC',
       'Cl[C@H]1[C@H](Cl)[C@@H](Cl)[C@@H](Cl)[C@H](Cl)[C@H]1Cl',
       'O=C([O-])[C@@H](O)[C@@H](O)[C@H](O)[C@@H](O)C(=O)[O-]', ...,
       'O=[Zn]', '[O-][Cl+3]([O-])([O-])O', 'S=[Se]=S'], dtype=object)

In [197]:
from SmilesPE.pretokenizer import kmer_tokenizer
import sys

corpus = []
for i in train:
    temp = spe.tokenize(i)
    temp = temp.strip().split(' ')
    corpus.append(temp)
print(corpus)

[['*', 'C(=O)[C@H](', 'CCCCNC(=O)', 'OCC', 'OC)', 'NC(=O)', 'OCC', 'OC'], ['Cl', '[C@H]1[C@H](', 'Cl)', '[C@@H](', 'Cl)', '[C@@H](', 'Cl)', '[C@H](', 'Cl)', '[C@H]1', 'Cl'], ['O=C(', '[O-])', '[C@@H](O)', '[C@@H](O)[C@H](O)', '[C@@H](O)', 'C(=O)', '[O-]'], ['[H]', '/', '[NH+]', '=C(\\', 'N)', 'C1=CC(=O)', '/C(=C\\', 'C=', 'c2ccc(', '=C(N)', '[NH3+]', ')cc2)', 'C=C1'], ['[H]', '/', '[NH+]', '=C(\\', 'N)', 'c1ccc(O', 'CCCC', 'COc2ccc(', '/C(', 'N)', '=', '[NH+]', '/', '[H]', ')cc2)cc1'], ['O=[N+]([O-])', '[O-]'], ['[N]', '=O'], ['O=', '[99Tc]', '(=O)', '(=O)', '[O-]'], ['O=', 'P', '([O-])', '([O-])', 'F'], ['O=', 'S(=O)', '([O-])', '[O-]'], ['O=S(', '[O-])', '([O-])', '=S'], ['[Se]'], ['CC(C)', 'C[C@H](', 'NC(=O)[C@H](', 'Cc1ccccc1)NC(=O)', 'c1cn', 'ccn1)', 'B(O)', 'O'], ['CC(C)', 'C[C@H](', 'NC(=O)[C@H](', 'Cc1ccccc1)NC(=O)', 'c1cn', 'ccn1)', 'B(O)', 'O'], ['CC(C)C[C@H](NC(=O)', 'CNC(=O)', 'c1cc(Cl)ccc1', 'Cl)', 'B(O)', 'O'], ['C#', 'CC1', '(O', 'C(N)=O)', 'CCCCC1'], ['C#', 'CC', '[NH2+

In [226]:

print("학습 중")
model = Word2Vec(corpus, vector_size=500, workers=4, sg=1, epochs=50, compute_loss=True,min_count=1, callbacks=[callback()])
print('완료')

학습 중
Loss after epoch 0: 152244.96875
Loss after epoch 1: 127708.03125
Loss after epoch 2: 104848.75
Loss after epoch 3: 98684.5
Loss after epoch 4: 95461.4375
Loss after epoch 5: 91276.25
Loss after epoch 6: 89005.3125
Loss after epoch 7: 86005.4375
Loss after epoch 8: 83521.6875
Loss after epoch 9: 82448.6875
Loss after epoch 10: 79001.5625
Loss after epoch 11: 77989.125
Loss after epoch 12: 75454.5
Loss after epoch 13: 74130.375
Loss after epoch 14: 72160.125
Loss after epoch 15: 71003.75
Loss after epoch 16: 69992.875
Loss after epoch 17: 68006.75
Loss after epoch 18: 66308.25
Loss after epoch 19: 65121.0
Loss after epoch 20: 64491.0
Loss after epoch 21: 62375.625
Loss after epoch 22: 61834.25
Loss after epoch 23: 61169.5
Loss after epoch 24: 59828.25
Loss after epoch 25: 59060.5
Loss after epoch 26: 55163.75
Loss after epoch 27: 55307.5
Loss after epoch 28: 53480.25
Loss after epoch 29: 53549.5
Loss after epoch 30: 53314.0
Loss after epoch 31: 52614.75
Loss after epoch 32: 52150.7

In [227]:
wv = model.wv

In [228]:
index = 0
for i in corpus:
    for j in i:
        try:
            wv[j]
        except:
            index+=1
            print(j)
print(index)

0


In [214]:
embeding_matrix = []
for sentence in corpus:
    
    for word in 

KeyError: "Key '*C(=O)[C@H](CCCCNC(=O)OCCOC)NC(=O)OCCOC' not present"

In [183]:
import codecs
from SmilesPE.tokenizer import *

spe_vob= codecs.open('SPE_ChEMBL.txt')
spe = SPE_Tokenizer(spe_vob)

smi = 'CC[N+](C)(C)Cc1ccccc1Br'
spe.tokenize(smi)


'CC [N+](C) (C)C c1ccccc1 Br'